In [5]:
!pip install -q kaggle

In [7]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [8]:
! cp kaggle.json ~/.kaggle/

In [9]:
! chmod 600 ~/.kaggle/kaggle.json

In [10]:
! kaggle datasets list

ref                                                                title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
arnabchaki/data-science-salaries-2023                              Data Science Salaries 2023 💸                       25KB  2023-04-13 09:55:16          10679        286  1.0              
asahu40/walmart-data-analysis-and-forcasting                       Walmart Data Analysis and Forcasting              122KB  2023-04-26 07:07:03           1481         35  1.0              
ahmedshahriarsakib/usa-real-estate-dataset                         USA Real Estate Dataset                             1MB  2023-04-30 03:51:36           1211         33  1.0              
iammustafatz/diabetes-prediction-dataset               

In [11]:
! kaggle datasets download -d dorianlazar/medium-articles-dataset

100% 1.32G/1.33G [00:25<00:00, 30.3MB/s]
100% 1.33G/1.33G [00:25<00:00, 56.6MB/s]


In [12]:
! kaggle datasets download -d dorianlazar/medium-articles-dataset

medium-articles-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [13]:
!unzip /content/medium-articles-dataset.zip -d dataset

Streaming output truncated to the last 5000 lines.
  inflating: dataset/images/2249.jpeg  
  inflating: dataset/images/225.png  
  inflating: dataset/images/2250.jpeg  
  inflating: dataset/images/2251.jpg  
  inflating: dataset/images/2252.jpeg  
  inflating: dataset/images/2253.jpeg  
  inflating: dataset/images/2254.jpg  
  inflating: dataset/images/2255.jpg  
  inflating: dataset/images/2256.png  
  inflating: dataset/images/2257.jpeg  
  inflating: dataset/images/2258.jpeg  
  inflating: dataset/images/2259.jpg  
  inflating: dataset/images/226.jpeg  
  inflating: dataset/images/2260.jpeg  
  inflating: dataset/images/2261.jpeg  
  inflating: dataset/images/2262.jpg  
  inflating: dataset/images/2263.jpg  
  inflating: dataset/images/2264.jpg  
  inflating: dataset/images/2265.jpeg  
  inflating: dataset/images/2266.jpg  
  inflating: dataset/images/2267.jpeg  
  inflating: dataset/images/2268.jpeg  
  inflating: dataset/images/2269.jpeg  
  inflating: dataset/images/227.jpg  
  i

In [1]:
import pickle 
import os
import pandas as pd
import numpy as np
import tensorflow as tf

In [14]:
medium_data = pd.read_csv('/content/dataset/medium_data.csv')
medium_data.head()

,id,url,title,subtitle,image,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30
1,2,https://towardsdatascience.com/hands-on-graph-...,Hands-on Graph Neural Networks with PyTorch & ...,NaN,2.png,1100,11,9,Towards Data Science,2019-05-30
2,3,https://towardsdatascience.com/how-to-use-ggpl...,How to Use ggplot2 in Python,A Grammar of Graphics for Python,3.png,767,1,5,Towards Data Science,2019-05-30
3,4,https://towardsdatascience.com/databricks-how-...,Databricks: How to Save Files in CSV on Your L...,When I work on Python projects dealing…,4.jpeg,354,0,4,Towards Data Science,2019-05-30
4,5,https://towardsdatascience.com/a-step-by-step-...,A Step-by-Step Implementation of Gradient Desc...,One example of building neural…,5.jpeg,211,3,4,Towards Data Science,2019-05-30


In [16]:
medium_data['title'] = medium_data['title'].apply(lambda x: x.replace(u'\xa0',u' '))
medium_data['title'] = medium_data['title'].apply(lambda x: x.replace('\u200a',' '))

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(oov_token='<oov>') 
tokenizer.fit_on_texts(medium_data['title'])
total_words = len(tokenizer.word_index) + 1

print("Total number of words: ", total_words)
print("Word: ID")
print("------------")
print("<oov>: ", tokenizer.word_index['<oov>'])
print("Strong: ", tokenizer.word_index['strong'])
print("And: ", tokenizer.word_index['and'])
print("Consumption: ", tokenizer.word_index['consumption'])

Total number of words:  8238
Word: ID
------------
<oov>:  1
Strong:  4
And:  8
Consumption:  8237


In [20]:
input_sequences = []
for line in medium_data['title']:
    token_list = tokenizer.texts_to_sequences([line])[0]
    #print(token_list)
    
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# print(input_sequences)
print("Total input sequences: ", len(input_sequences))

Total input sequences:  48461


In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   5, 676,
        68], dtype=int32)

In [22]:
x, labels = input_sequences[:,:-1],input_sequences[:,-1]
y = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [23]:
print(x[5])
print(labels[5])
print(y[5][14])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    5  676   68    2  452 1518]
14
1.0


In [2]:
pickled_model = pickle.load(open('model.pkl', 'rb'))

In [25]:
text = "My favourite book"
next_words = 20
  
for i in range(next_words):
    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predict_x=pickled_model.predict(token_list) 
    predicted=np.argmax(predict_x,axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    text += " " + output_word
print(text)

1/1 [==============================] - 0s 218ms/step
My favourite book writing for the abusive flow of rejection made in the attempt decade behind with unimportant tasks rests fail in monetary
